# notebook to post-process results from SLEAP into  freemocap format

expected processing steps:

1 - load data from SLEAP h5's (one per video, produced by SLEAP's GUI process)
2 - Reformat into standard freemocap 2d npy format (dimensions: camera, frame, marker, XY)
2a,b - save and test by producing annotated videos
3 - triangulate using anipose triangulation
4 - save triangulation results to freemocap formated 3d npy  (frame, marker, XYZ) (and another npy for reprojection error)

In [ ]:
import numpy as np
import h5py
from pathlib import Path
from aniposelib.cameras import CameraGroup


In [ ]:
session_id = 'sesh_2022-05-07_17_15_05_pupil_wobble_juggle_0'

freemocap_data_folder = Path('C:/Users/jonma/Dropbox/FreeMoCapProject/FreeMocap_Data')

session_folder_path = freemocap_data_folder / session_id

calibration_file_name = session_id + '_calibration.toml'
camera_calibration_file_path = session_folder_path / calibration_file_name

sleap_data_path = session_folder_path / 'sleap_data'
sleap_h5_files_path = sleap_data_path / 'output_h5_files'

sleap_predictions_file = sleap_data_path / 'labels.v000.slp'

In [ ]:
anipose_camera_calibration_object = CameraGroup.load(camera_calibration_file_path)

In [ ]:


per_video_labels_fr_point_xy = []
per_video_label_names = []
for this_h5_file in sleap_h5_files_path.glob('*.h5'):
    with h5py.File(this_h5_file, "r") as f:
        dset_names = list(f.keys())
        locations = f["tracks"][:].T
        node_names = [n.decode() for n in f["node_names"][:]]

    per_video_labels_fr_point_xy.append(locations)
    per_video_label_names.append(node_names)
    print("===filename===")
    print(this_h5_file)
    print()

    print("===HDF5 datasets===")
    print(dset_names)
    print()

    print("===locations data shape===")
    print(locations.shape)
    print()

    print("===nodes===")
    for i, name in enumerate(node_names):
        print(f"{i}: {name}")
    print()




In [ ]:
number_of_videos = len(per_video_labels_fr_point_xy)
number_of_frames = len(per_video_labels_fr_point_xy[0])
number_of_markers = len(per_video_labels_fr_point_xy[0][0])
sleap2d_nCams_nFrames_nImgPts_XY = np.empty((number_of_videos, number_of_frames, number_of_markers, 2))

put data into freemocap 2d_npy format

In [ ]:
for this_video_number in range(len(per_video_labels_fr_point_xy)):
    sleap2d_nCams_nFrames_nImgPts_XY[this_video_number,:,:,:] = np.squeeze(per_video_labels_fr_point_xy[this_video_number])

sleap2d_nCams_nFrames_nImgPts_XY.shape

load anipose calibration cgroup whosit

In [ ]:

sleap_flattened_nCams_nTotalPoints_XY = sleap2d_nCams_nFrames_nImgPts_XY.reshape(number_of_videos, -1, 2)  # reshape data to collapse across 'frames' so it becomes [numCams, numFrames*numPoints, XY]

print('Reconstructing 3d points...')
sleap_data3d_flat = anipose_camera_calibration_object.triangulate(sleap_flattened_nCams_nTotalPoints_XY, progress=True)

sleap_data_reprojerr_flat = anipose_camera_calibration_object.reprojection_error( sleap_data3d_flat, sleap_flattened_nCams_nTotalPoints_XY, mean=True)

##return:
sleap_fr_mar_xyz = sleap_data3d_flat.reshape(number_of_frames, number_of_markers, 3)
sleap_fr_mar_reprojectionError = sleap_data_reprojerr_flat.reshape(number_of_frames, number_of_markers)


In [ ]:

print(f'sleap_fr_mar_xyz.shape: {sleap_fr_mar_xyz.shape}')
print(f'sleap_fr_mar_reprojectionError.shape: {sleap_fr_mar_reprojectionError.shape}')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib 


In [ ]:
plt.close('all')
fig  = plt.figure()

for this_marker_number in range(sleap_fr_mar_xyz.shape[1]):
    ax = fig.add_subplot(5,2,this_marker_number+1)
    ax.plot(sleap_fr_mar_xyz[:,this_marker_number,:])

save sleap data to `DataArrays` folder 

In [ ]:
session_data_arrays_path = session_folder_path / 'DataArrays'

sleap_3d_save_path = session_data_arrays_path / 'sleap_3d_points.npy'
sleap_reproj_err_save_path = session_data_arrays_path / 'sleap_reprojection_error.npy'
label_name_csv_path = session_data_arrays_path / 'sleap_label_names.csv'

np.save(sleap_3d_save_path, sleap_fr_mar_xyz)
np.save(sleap_reproj_err_save_path, sleap_fr_mar_reprojectionError)

import csv
with open(label_name_csv_path,'w') as label_name_file:
    wr = csv.writer(label_name_file)
    wr.writerow(node_names)

put data into freemocap 2d_npy format

In [12]:
for this_video_number in range(len(per_video_labels_fr_point_xy)):
    sleap2d_nCams_nFrames_nImgPts_XY[this_video_number,:,:,:] = np.squeeze(per_video_labels_fr_point_xy[this_video_number])

sleap2d_nCams_nFrames_nImgPts_XY.shape

(5, 5090, 10, 2)

load anipose calibration cgroup whosit

In [13]:

sleap_flattened_nCams_nTotalPoints_XY = sleap2d_nCams_nFrames_nImgPts_XY.reshape(number_of_videos, -1, 2)  # reshape data to collapse across 'frames' so it becomes [numCams, numFrames*numPoints, XY]

print('Reconstructing 3d points...')
sleap_data3d_flat = anipose_camera_calibration_object.triangulate(sleap_flattened_nCams_nTotalPoints_XY, progress=True)

sleap_data_reprojerr_flat = anipose_camera_calibration_object.reprojection_error( sleap_data3d_flat, sleap_flattened_nCams_nTotalPoints_XY, mean=True)

##return:
sleap_fr_mar_xyz = sleap_data3d_flat.reshape(number_of_frames, number_of_markers, 3)
sleap_fr_mar_reprojectionError = sleap_data_reprojerr_flat.reshape(number_of_frames, number_of_markers)


Reconstructing 3d points...


100%|█████████████████████████| 50900/50900 [00:07<00:00, 6795.51it/s]


In [14]:

print(f'sleap_fr_mar_xyz.shape: {sleap_fr_mar_xyz.shape}')
print(f'sleap_fr_mar_reprojectionError.shape: {sleap_fr_mar_reprojectionError.shape}')

sleap_fr_mar_xyz.shape: (5090, 10, 3)
sleap_fr_mar_reprojectionError.shape: (5090, 10)


In [15]:
import matplotlib.pyplot as plt
%matplotlib 


Using matplotlib backend: TkAgg


In [22]:
plt.close('all')
fig  = plt.figure()

for this_marker_number in range(sleap_fr_mar_xyz.shape[1]):
    ax = fig.add_subplot(5,2,this_marker_number+1)
    ax.plot(sleap_fr_mar_xyz[:,this_marker_number,:])

save sleap data to `DataArrays` folder 

In [ ]:
session_data_arrays_path = session_folder_path / 'DataArrays'

sleap_3d_save_path = session_data_arrays_path / 'sleap_3d_points.npy'
sleap_reproj_err_save_path = session_data_arrays_path / 'sleap_reprojection_error.npy'
label_name_csv_path = session_data_arrays_path / 'sleap_label_names.csv'

np.save(sleap_3d_save_path, sleap_fr_mar_xyz)
np.save(sleap_reproj_err_save_path, sleap_fr_mar_reprojectionError)

import csv
with open(label_name_csv_path,'w') as label_name_file:
    wr = csv.writer(label_name_file)
    wr.writerow(node_names)